In [8]:
import pandas as pd
from time import sleep
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re

# Read in address data<br>
### Note that this code will only work if the correct way to replace spaces in the Address column is with the following string: <i>'%20'</i><br>
### Try searching Google maps for an address with spaces and it will automatically fill in the spaces using some string, so just figure out what it is and use it below

In [9]:
# set the 'space filler' that Google Maps uses on a browser, then read in address data
space_filler = '%20'
df = pd.read_csv('Forsyth_2023.csv')

df['full_address'] = df['Address'] + ' Forsyth County, GA'

# # take random sample from initial CSV
# df = df.sample(n = 1000)
df

,Parcel ID,Address,Owner Name,Owner Address,Grantor,Grantee,Sale Date,Sale Price,Qualified Sales,Sales Validity,Acres,Parcel Class,Tax District,Year Built,Square Ft,Neighborhood,Zoning,full_address
0,035 425,6110 FORMOSA LN,MCDEERMOND RYAN & | MCDEERMOND JENNIFER,MCDEERMOND RYAN & | MCDEERMOND JENNIFER | 6110...,MCDEERMOND RYAN & MCDEERMOND JENNIFER,ORCHARD PROPERTY III LLC,2/27/23,"$511,350.00",Unqualified,Not Subject to Open Market,0.24,R3,0,2014,3068,795,RES3,"6110 FORMOSA LN Forsyth County, GA"
1,065 178,6940 WHITMAN WALK,EA HOMES LP,EA HOMES LP | 5775 GLENRIDGE DR BLDG D STE 350...,EA HOMES LP,STOCKS THOMAS F,2/10/23,"$822,440.00",Unqualified,UNKNOWN,0.08,R3,0,2022,2304,25195,MP,"6940 WHITMAN WALK Forsyth County, GA"
2,035 117,6430 IVEY CREEK CIR,ROBERTS JANAE G,ROBERTS JANAE G | 417 ATLAS ST | WHITE SANDS M...,ROBERTS JANAE G,FLITCH PROPERTIES LLC,2/8/23,"$397,000.00",Qualified,NaN,0.65,R3,0,1999,2521,438,R2R,"6430 IVEY CREEK CIR Forsyth County, GA"
3,075 290,3720 BEAUPORT DR,BHEG MONTEBELLO LLC,BHEG MONTEBELLO LLC | 1140 VIRGINIA DR | FORT ...,BHEG MONTEBELLO LLC,TOLL SOUTHEAST LP COMPANY INC,2/1/23,"$1,681,073.00",Unqualified,Sale Includes more than one parcel,0.28,R3,0,2022,3583,260,RES3,"3720 BEAUPORT DR Forsyth County, GA"
4,083 598,7030 MAJORS RD,ZB HOMES AT MAJORS ROAD LLC,ZB HOMES AT MAJORS ROAD LLC | 202 TRIBBLE GAP ...,ZB HOMES AT MAJORS ROAD LLC,NYCZ TYLER &,2/3/23,"$759,900.00",Unqualified,UNKNOWN,1.01,R3,0,2022,3215,25233,RES1,"7030 MAJORS RD Forsyth County, GA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,249 070,5290 RIDGE FARMS DR,MCMAHON JOHN TIMOTHY & | MCMAHON ANGELA ANDERSON,MCMAHON JOHN TIMOTHY & | MCMAHON ANGELA ANDERS...,MCMAHON JOHN TIMOTHY &,GIBSON MARK DAVIS &,3/1/23,"$601,000.00",Qualified,NaN,1.33,R3,0,2006,3543,2127,R3,"5290 RIDGE FARMS DR Forsyth County, GA"
1271,250 092,6785 PAYTON RD,OPENDOOR PROPERTY TRUST 1,OPENDOOR PROPERTY TRUST 1 | 410 N SCOTTSDALE R...,OPENDOOR PROPERTY TRUST I,KHALAF DALIA,3/1/23,"$400,000.00",Qualified,NaN,0.59,R3,0,1994,1504,2103,R2R,"6785 PAYTON RD Forsyth County, GA"
1272,174 222,1435 LOBLOLLY LN,BI KEPING & | ZHOU YE,BI KEPING & | ZHOU YE | 705 EMORY BLUFF | JOHN...,BI KEPING & ZHOU YE,RAMSEY CONNOR &,3/1/23,"$525,000.00",Qualified,NaN,0.70,R3,0,1994,2460,2003,R2R,"1435 LOBLOLLY LN Forsyth County, GA"
1273,283 120,7315 TIMBERLINE OVLK,PRESTON JOSHUA I & | PRESTON BETHANY N,PRESTON JOSHUA I & | PRESTON BETHANY N | 7315 ...,PRESTON JOSHUA I & PRESTON BETHANY N,HIGGINS BRENDAN JAMES &,3/1/23,"$430,000.00",Unqualified,UNKNOWN,0.57,R3,0,1999,2373,2217,R2R,"7315 TIMBERLINE OVLK Forsyth County, GA"


In [10]:
df['Address_URL'] = df['full_address'].str.replace(' ', space_filler)

# creates the url which will be used to return the response below
df['url'] = ['https://www.google.com/maps/search/' + i for i in df['Address_URL']]
df = df[['full_address', 'url']]
df.head(10)

,full_address,url
0,"6110 FORMOSA LN Forsyth County, GA",https://www.google.com/maps/search/6110%20FORM...
1,"6940 WHITMAN WALK Forsyth County, GA",https://www.google.com/maps/search/6940%20WHIT...
2,"6430 IVEY CREEK CIR Forsyth County, GA",https://www.google.com/maps/search/6430%20IVEY...
3,"3720 BEAUPORT DR Forsyth County, GA",https://www.google.com/maps/search/3720%20BEAU...
4,"7030 MAJORS RD Forsyth County, GA",https://www.google.com/maps/search/7030%20MAJO...
5,"4825 ASCOT DR Forsyth County, GA",https://www.google.com/maps/search/4825%20ASCO...
6,"3530 CHARLESBOURG RUN Forsyth County, GA",https://www.google.com/maps/search/3530%20CHAR...
7,"3285 CARSWELL BND Forsyth County, GA",https://www.google.com/maps/search/3285%20CARS...
8,"3555 CHARLESBOURG RUN Forsyth County, GA",https://www.google.com/maps/search/3555%20CHAR...
9,"3265 CARSWELL BND Forsyth County, GA",https://www.google.com/maps/search/3265%20CARS...


# Geocode

In [ ]:
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

results = []

for ind in tqdm(df.index, colour='#00FFFF', desc='Geocoding Progress'):
    try:
        driver.get(df['url'][ind])
        sleep(3.7)
        url = driver.current_url
        results.append(url)
    except:
        results.append('error')

print('Geocoding complete!')

Geocoding Progress:   4%|▊                  | 54/1275 [04:04<1:32:27,  4.54s/it]

# Create new columns, clean up dataframe, output summary

In [11]:
lats = []
longs = []

# parse & split the 'results' list 
for item in range(len(results)):
    try:
        found = re.search('/@(.+?),17z', results[item]).group(1)
        lats.append(found.split(',')[0])
        longs.append(found.split(',')[1])
    except:
        lats.append('error')
        longs.append('error')
        
# now add the parsed & cleaned lat/longs as additional columns 
df = df.copy()
df['lat'] = lats
df['long'] = longs
# df = df.drop(columns=['Address_URL', 'url'])
df

,full_address,url,lat,long
40367,736 Independant Ave PENDERGRASS Georgia,https://www.google.com/maps/search/736%20Indep...,34.1473932,-83.6687729
41956,73 TIPPERARY CT JEFFERSON Georgia,https://www.google.com/maps/search/73%20TIPPER...,34.1029608,-83.6778704
88848,6014 THORNLAKE DR FLOWERY BRANCH Georgia,https://www.google.com/maps/search/6014%20THOR...,34.1928714,-83.9475294
10165,1281 OVERLOOK RIDGE RD BISHOP Georgia,https://www.google.com/maps/search/1281%20OVER...,33.8578867,-83.5254898
53510,5310 Falls Dr Apt 11001 CUMMING Georgia,https://www.google.com/maps/search/5310%20Fall...,34.2999385,-84.0742251
...,...,...,...,...
103269,153 BO DIG CIR DAHLONEGA Georgia,https://www.google.com/maps/search/153%20BO%20...,34.591002,-84.009463
42675,1327 TAPP WOOD RD HOSCHTON Georgia,https://www.google.com/maps/search/1327%20TAPP...,34.0663895,-83.707231
53438,62 Land Way COMMERCE Georgia,https://www.google.com/maps/search/62%20Land%2...,34.2108728,-83.4503702
70576,4901 Bryant Quarter Rd GILLSVILLE Georgia,https://www.google.com/maps/search/4901%20Brya...,34.273503,-83.6523193


In [10]:
# how many addresses will need to be manually geocoded?
manuals = df[df['lat'] == 'error'].shape[0]
total_addresses = df.shape[0]
perc = (manuals / total_addresses) * 100
print(f'Addresses to manually geocode: {manuals} out of {total_addresses}, or only {perc:.1f}% of the total!')

KeyError: "['Address_URL'] not found in axis"

# If everything looks good, export to CSV

In [168]:
df.to_csv('GeocodedAddresses.csv')

Export complete
